In [219]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

DATA ANALYSIS

In [220]:
#reading data
traindf=pd.read_csv("/kaggle/input/titanic/train.csv")
testdf=pd.read_csv("/kaggle/input/titanic/test.csv")
finaldf=pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [221]:
traindf.head()

**Checking missing values in training data**

In [222]:
traindf.isnull().sum()

**Handling missing values in training data:**
for numeric column like Age,fill the missing values with median and for categorical columns with mode.Since there can be more than 1 mode in a data,I filled it with the first value in the mode.

In [223]:
traindf.loc[traindf["Age"].isna()==True,"Age"]=traindf["Age"].median()
traindf.loc[traindf["Cabin"].isna()==True,"Cabin"]=traindf["Cabin"].mode()[0]
traindf.loc[traindf["Embarked"].isna()==True,"Embarked"]=traindf["Embarked"].mode()[0]

In [224]:
traindf.isnull().sum()

In [225]:
testdf.isnull().sum()

**Handling missing values in testing data**

In [226]:
testdf.loc[testdf["Age"].isna()==True,"Age"]=testdf["Age"].median()
testdf.loc[testdf["Fare"].isna()==True,"Fare"]=testdf["Fare"].median()
testdf.loc[testdf["Cabin"].isna()==True,"Cabin"]=testdf["Cabin"].mode()[0]

In [227]:
testdf.isna().sum()

**Separating numeric and categorical columns**

In [228]:
numdf=traindf.select_dtypes(["int64","float64"])
catdf=traindf.select_dtypes(["object"])

In [229]:
numdf.head()

In [230]:
catdf.head()

In [231]:
catdf.drop(columns=["Name","Ticket","Cabin"],inplace=True)

In [232]:
catdf["Sex"]=catdf["Sex"].map({"male":0,"female":1})
catdf.head()

In [233]:
catdf["Embarked"].value_counts()

In [234]:
catdf["Embarked"]=catdf["Embarked"].map({"S":0,"C":1,"Q":2})
catdf.head()

In [235]:
traindf2=pd.concat([numdf,catdf],axis=1)
traindf2.head()

In [236]:
a=list(traindf2.columns)
a.remove("Survived")
testdf2=testdf[a]
testdf2.head()

In [237]:
testdf2["Embarked"]=testdf2["Embarked"].map({"S":0,"C":1,"Q":2})
testdf2["Sex"]=testdf2["Sex"].map({"male":0,"female":1})
testdf2.head()

**heatmap to test the relationship between columns**

In [238]:
plt.figure(figsize=(20,7))
sns.heatmap(traindf2.corr(),annot=True)
plt.show()

**columns with very less relationship with survived:**
PassengerId,Age,SibSp and Parch


In [239]:
#deleting the columns with very less relationship
xtrain=traindf2[["Pclass","Fare","Sex","Embarked"]]
ytrain=traindf2["Survived"]

**Predictions using Random Forest classifier**

In [240]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_jobs=-1)
rfc.fit(xtrain,ytrain)
rfc.score(xtrain,ytrain)

In [241]:
ytrain_pred=pd.Series(rfc.predict(xtrain))
ytrain_pred.value_counts()

**Model Evaluation using confusion matrix and classification report**

In [242]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(ytrain,ytrain_pred))
print(classification_report(ytrain,ytrain_pred))

The model looks decent.

In [243]:
ytestpred=pd.Series(rfc.predict(testdf2[xtrain.columns]))
ytestpred.value_counts()

In [244]:
print(confusion_matrix(finaldf.Survived,ytestpred))
print(classification_report(finaldf.Survived,ytestpred))

In [245]:
df=pd.concat([testdf.PassengerId,ytestpred],axis=1)
df.rename(columns={0:"Survived"},inplace=True)
df.head()

In [246]:
df.to_csv("recpredictions.csv")